<a href="https://colab.research.google.com/github/yeb2Binfang/ECE-GY-9163-ML-cyber/blob/main/Lab/LR_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Logistic regression review

LR 是很重要的模型，教授在课上也给我们认真地讲过了，这里就把过程以及推论都给详细地写一下

LR是一个很经典的二分类的分类方法，在工业界中，LR也是经常使用的，非常重要，面试也是经常考的，所以我们要去了解怎么推导LR，了解LR的objective function，了解在LR模型下怎么避免overfitting，LR的框架。LR一般都可以作为baseline

在很多场景下都是可以用LR的，比如说
* 贷款违约情况(会/不会)
* 广告点击问题，也就是CTR prediction
* 商品推荐
* 情感分析
* 疾病诊断
* 等等

我们来看一个例子，是一个classification problem，这是一个二分类的问题

现在有这么一个数据

| Age      | Salary | Education      | If Breach of contract |
| :----: | :----: | :----: | :----: |
| 20      | 4000       | undergrad      | Yes     |
| 25   | 5000        | college      | No      |
| 21      | 6000       | undergrad     | No      |
| 25   | 5000       | college     | Yes      |
| 28      | 8000      | undergrad      | No       |
| 27  | 7000        | undergrad      | ?      |

学习输入到输出的映射 $f:X→ Y$

$X$就是我们的属入

$Y$就是我们的输出

这里呢，其实我们也就是想求这么一个概率，也就是$P(Y|X)$。

那我们现在思考一个问题，就是我们可不可以用线性回归来表示$P(Y|X)=w^Tx+b$? 为什么？

当然是不可以了
* 因为$P(Y|X)$是条件概率，它的范围是[0,1]
* 其次就是所有类的概率加起来是1，也就是$∑_y P(Y|X)=1$.
* 而线性回归的范围是(-∞,+∞)

那我们可不可以把线性回归的$w^Tx+b$给映射到[0,1]呢？于是我们就想到了logistic function，也就是
$$
y=\frac{1}{1+e^{-z}}, z=w^Tx+b
$$
它的图像如下图所示

<img src="https://user-images.githubusercontent.com/68700549/147372757-b5acd7b2-7096-4401-b178-bc4bd07e9ce1.png" width="300" height="200" />





